In [2]:
import severability
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("Lightcast_co_mat.csv", index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3906 entries, Dc.js to Dyeing
Columns: 3906 entries, Dc.js to Dyeing
dtypes: float64(3906)
memory usage: 116.4+ MB


In [4]:
labels = df.columns
A = np.asmatrix(df.fillna(0).values)
P = severability.transition_matrix(A)
P

matrix([[4.86233750e-06, 5.39719463e-04, 5.34857125e-05, ...,
         4.86233750e-05, 8.26597375e-06, 0.00000000e+00],
        [2.35547165e-04, 1.48118848e-04, 1.84405844e-04, ...,
         2.14326700e-05, 1.74007816e-05, 0.00000000e+00],
        [1.79542210e-05, 1.41838346e-04, 1.30804661e-03, ...,
         3.91728458e-06, 3.26440381e-07, 0.00000000e+00],
        ...,
        [4.10052856e-04, 4.14153384e-04, 9.84126854e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [5.13285024e-05, 2.47584541e-04, 6.03864734e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [5]:
class Skill:
    def __init__(self, name, index):
        self.name = name
        self.index = index

    def __repr__(self):
        comp = getattr(self, 'component', 'not calculated yet')
        return f"Skill(name={self.name}, index={self.index}, component={comp})"
    
    def calculate_components(self):
        print(f"Calculating components for {self.name} at index {self.index}")
        components = []
        for t in range(1, 3):
            comp = severability.node_component(P, self.index, t, max_size=5)[0]
            print(f"Component for t={t}: {list(map((lambda x: labels[x]), comp))}")
            components.append(comp)
        self.components = components

In [6]:
# Load and clean the skills taxonomy
skills = pd.read_csv("Lightcast_taxonomy_M2R.csv", index_col=1)
if 'Unnamed: 0' in skills.columns:
    skills = skills.drop('Unnamed: 0', axis=1)

# Function to get skill:index mapping for a category, filtered by available labels
def get_skills_from_category(category, labels):
    # Get all skills in the category
    category_skills = skills[skills['Category (corrected)'] == category].index
    
    # Intersect with labels (preserves order and position in labels)
    filtered_labels = labels.intersection(category_skills)
        
    # Build mapping: skill_name -> index in labels
    return [ Skill(skill, labels.get_loc(skill)) for skill in filtered_labels ]

tech_skills = get_skills_from_category('Information Technology', labels) + get_skills_from_category('Analysis', labels)
hospitality_skills = get_skills_from_category('Hospitality and Food Services', labels)

len(tech_skills), len(hospitality_skills)

(834, 48)

In [19]:
D = A * np.ones(A.shape[0])

ValueError: shapes (3906,3906) and (1,3906) not aligned: 3906 (dim 1) != 1 (dim 0)